In [56]:
%matplotlib inline
import numpy as np
from sklearn.preprocessing import LabelEncoder
import mne
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_selection import RFECV

## Análise  da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a (1) extração de características. (2) seleção de características e (3) classificação dos dados.

### Pontos Importantes do Dataset
 
- Frequências estimuladas (total de 40, com a diferença de 0.2 Hz uma da outra): 8.0, 8.2, ..., 15.6, 15.8;
- Taxa de amostragem: 250 Hz


### Análisar os "momentos" em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados do participante.
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 - 18 Hz.
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo).
   - **a)** 0.0 - 0.5 segundos e  2.5 - 3.0 segundos ocorre apenas ruído;
   - **b**) 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos)
4. Com os sinais separados em objetos MNE, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações. 
   - Os dados devem ser plotados no dominio da frequência (após a transformada de Fourier). O FFT pode ser realizada pela biblioteca `scipy.fft`.
   - Deve ser observado que as janelas (a) com ruídos não aparecerão de fato o sinal SSVEP.

In [57]:
data = np.load("../../datasets/beta/data.npy")

print(data.shape)

(160, 64, 750)


In [58]:
n_channels = 64
sfreq = 250
ch_names = list(np.load("../../datasets/beta/channels.npy"))
ch_types = ['eeg'] * len(ch_names)
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)
best = ['P6', 'PO3']

In [59]:
labels = np.load("../../datasets/beta/labels.npy")
unique_labels = sorted(set(labels))
event_dict = {str(value): index  for index, value in enumerate(unique_labels)}
labels.shape

(160,)

In [60]:
# método para transformar labels categóricos
le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(160, dtype=int),
    le.fit_transform(labels))
)

# print(events.shape)

mne_data = mne.EpochsArray(data, info, events, event_id=event_dict)
filtered_mne_data = mne_data.filter(6, 18)

filtered_mne_data

Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 6 - 18 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 6.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Upper passband edge: 18.00 Hz
- Upper transition bandwidth: 4.50 Hz (-6 dB cutoff frequency: 20.25 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 3527 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done 4607 tasks      | elapsed:    1.0s
[Parallel(n_job

Number of events,160
Events,10.0: 410.2: 410.4: 410.6: 410.8: 411.0: 411.2: 411.4: 411.6: 411.8: 412.0: 412.2: 412.4: 412.600000000000001: 412.8: 413.0: 413.200000000000001: 413.4: 413.600000000000001: 413.8: 414.0: 414.200000000000001: 414.4: 414.600000000000001: 414.8: 415.0: 415.200000000000001: 415.4: 415.600000000000001: 415.8: 48.0: 48.2: 48.4: 48.6: 48.799999999999999: 49.0: 49.2: 49.4: 49.6: 49.8: 4
Time range,0.000 – 2.996 s
Baseline,off


In [61]:
start_noise = filtered_mne_data.copy().crop(tmin=0.0, tmax=0.5)
middle = filtered_mne_data.copy().crop(tmin=0.5, tmax=2.5)
end_noise = filtered_mne_data.copy().crop(tmin=2.5, tmax=3.0)

noise_power = []

fft_start_result = np.fft.fft(start_noise)
fft_end_result = np.fft.fft(end_noise)

# densidade espectral de potência (PSD)
psd_start = np.abs(fft_start_result) ** 2
psd_end = np.abs(fft_end_result) ** 2

# média da potência nos intervalos de tempo sem estímulo
base_power = np.mean(psd_start, axis=-1)
rest_power = np.mean(psd_end, axis=-1)

# média das duas médias de potência obtidas anteriorment
mean_noise_power = np.mean([base_power + rest_power])

mean_noise_power

/tmp/ipykernel_7720/3614421376.py:3: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsArray'>.tmax (2.996 s)
  end_noise = filtered_mne_data.copy().crop(tmin=2.5, tmax=3.0)


5896.091879334966

In [62]:
sr = 250

# frequências alvo
target_frequencies = np.arange(8, 16, 0.2)
# lista para armazenar as amplitudes nas frequências alvo
target_amplitudes = []

for channel_data in middle.get_data():
    target = []
    for electrode in channel_data:

        fft_result = np.fft.fft(electrode)
        psd = np.abs(fft_result) ** 2
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
        target_amplitudes_trial = []
        for target_frequency in target_frequencies:
            # encontrando o índice da frequência alvo no espectro de frequência
            index = np.argmin(np.abs(frequencies - target_frequency))
            # amplitude na frequência alvo
            amplitude = np.sqrt(psd[index])
            target_amplitudes_trial.append(amplitude)

        target.append(target_amplitudes_trial)


    target_amplitudes.append(target)

target_amplitudes = np.array(target_amplitudes)

In [63]:
# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
target_amplitudes_adjusted = np.abs(target_amplitudes - mean_noise_power)

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / mean_noise_power)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)

print(target_amplitudes_adjusted.shape)

data = np.array([target_amplitudes_adjusted, narrow_band_SNR, wide_band_SNR])
data = data.swapaxes(0, 1)
data = data.swapaxes(1, 2)
data = data.reshape(data.shape[0], data.shape[1], data.shape[2] * data.shape[3])
print(data.shape)

(160, 64, 40)
(160, 64, 120)


### Primeira alternativa (remoção manual de características)

In [64]:
# Removendo eletrodos não listados

ch_names = np.load("../../datasets/beta/channels.npy")

channels_to_keep = ['PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'POZ', 'O1', 'OZ', 'O2']
indexes_to_remove = np.where(np.isin(ch_names, channels_to_keep, invert=True))

print("Before", data.shape)
data_filtered = np.delete(data, indexes_to_remove, axis=1)
print("After", data_filtered.shape)
print("Labels", labels.shape)

Before (160, 64, 120)
After (160, 9, 120)
Labels (160,)


In [65]:
data_filtered_reshape = data_filtered.reshape(data_filtered.shape[0], data_filtered.shape[1] * data_filtered.shape[2])
print(data_filtered_reshape.shape)

(160, 1080)


In [66]:
X = StandardScaler().fit_transform(data_filtered_reshape)
y = LabelEncoder().fit_transform(labels)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print("accuracy", accuracy_score(y_test, y_pred))
print("f1_score", f1_score(y_test, y_pred, average="weighted"))

accuracy 0.25
f1_score 0.2284722222222222


### Segunda alternativa (utilizando remoção automático de características RFE)

In [67]:
# reorganizando os dados

X_reshape_auto = data.reshape(data.shape[0], data.shape[1] * data.shape[2])
print(X_reshape_auto.shape)

(160, 7680)


In [68]:
X_reshape_auto = StandardScaler().fit_transform(X_reshape_auto)

rfe = RFECV(SVC(kernel="linear"), step=0.0001, min_features_to_select=1, cv=3)
X_final = rfe.fit_transform(X_reshape_auto, y)
print(X_final.shape)


KeyboardInterrupt: 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print("accuracy", accuracy_score(y_test, y_pred))
print("f1_score", f1_score(y_test, y_pred, average="weighted"))

accuracy 0.4583333333333333
f1_score 0.40972222222222227


#### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).

Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `SNR` (obrigatória);
- Maior valor espectral (FFT);
- Médio dos valores espectrais (FFT);


Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 channels e 750 values.

`160, 64 (SRN) + 64 (MÉDIA) + 64 (MAIOR) ...`

Resultando em  `160, 192`

#### Seleção de características e classsificação

Como existem diversos eletrodos (canais) que não obtem sinal SSVEP, podemos extrair as características que não contribuem para a classificação dos dados.


Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicador por meio de `sklearn.feature_selection.RFE`, aprimorand o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.

# How to calculate SNR https://saturncloud.io/blog/calculating-signaltonoise-ratio-in-python-with-scipy-v11/